In [174]:
from typing import List, Dict

import numpy as np
from mdptoolbox import mdp
from itertools import product, repeat
from functools import reduce
from operator import iconcat
from scipy.sparse import csr_matrix

from copy import deepcopy

In [175]:
# data
field_content = ['empty', 'white', 'blue', 'red']
task = ['store', 'restore']
item = ['white', 'blue', 'red']
task_and_item = [['store', 'white'], ['store', 'blue'], ['store', 'red'],
                 ['restore', 'white'], ['restore', 'blue'], ['restore', 'red']]
actions = [0, 1, 2, 3]

"""
2x2 warehouse with reward/distance at every position:

                [3/4 1/6]
in/out_item ->  [5/2 3/4]
"""
rewards_dict = {0: 5, 1: 3, 2: 3, 3: 1}
dist_dict = {0: 2, 1: 4, 2: 4, 3: 6}

In [176]:
def probabilities_from_data() -> Dict:
    f = open('trainingdata.txt', 'r')

    color_and_tasks = []
    line = f.readline()
    while line:
        color_and_tasks.append(line.split())
        line = f.readline()

    total_length = len(color_and_tasks)

    p_store_white = len([cur for cur in color_and_tasks if cur == ['store', 'white']]) / total_length
    p_store_blue = len([cur for cur in color_and_tasks if cur == ['store', 'blue']]) / total_length
    p_store_red = len([cur for cur in color_and_tasks if cur == ['store', 'red']]) / total_length
    p_restore_white = len([cur for cur in color_and_tasks if cur == ['restore', 'white']]) / total_length
    p_restore_blue = len([cur for cur in color_and_tasks if cur == ['restore', 'blue']]) / total_length
    p_restore_red = len([cur for cur in color_and_tasks if cur == ['restore', 'red']]) / total_length

    np.testing.assert_equal(p_store_white + p_store_blue + p_store_red + p_restore_white + p_restore_blue + p_restore_red,
                            1)

    return {'store_white': p_store_white,
            'store_blue': p_store_blue,
            'store_red': p_store_red,
            'restore_white': p_restore_white,
            'restore_blue': p_restore_blue,
            'restore_red': p_restore_red}


probs = probabilities_from_data()
print(probs)

{'store_white': 0.1278493557978196, 'store_blue': 0.12528906508093823, 'store_red': 0.24686157912124215, 'restore_white': 0.1278493557978196, 'restore_blue': 0.12528906508093823, 'restore_red': 0.24686157912124215}


In [177]:
def all_repeat() -> List:
    """
    Computes all permutations of items with repetitions

    :return: permutations of items with repetitions
    """
    result = []
    for c in product(field_content, repeat=4):
        comb = [c[0], c[1], c[2], c[3]]
        result.append(comb)

    result = list(repeat(result, 6))
    result = list(map(list, zip(result, task_and_item)))

    for i in range(len(result)):
        result[i] = [elem + result[i][1] for elem in result[i][0]]

    result = reduce(iconcat, result, [])

    return result

In [178]:
states = all_repeat()
num_states = len(states)

# check if all elements are unique.
np.testing.assert_equal(len(list(set(map(tuple, states)))), num_states)

In [179]:
def field_content_equals(from_state: list, to_state: list) -> bool:
    return from_state[:4] == to_state[:4]

In [180]:
def transition_prob(action: int, from_state: list, to_state: list) -> float:
    copy_from_state = deepcopy(from_state)
    next_item = to_state[-1]

    if copy_from_state[4] == 'store':
        if copy_from_state[action] != 'empty':
            return 1 if from_state == to_state else 0

        # from_state = [empty empty red empty] -> [red empty red empty]
        # to_state = [red empty red empty]
        copy_from_state[action] = copy_from_state[-1]
        return probs['store_' + next_item] if field_content_equals(copy_from_state, to_state) else 0
    elif copy_from_state[4] == 'restore':
        if copy_from_state[action] != copy_from_state[-1]:
            return 1 if from_state == to_state else 0

        copy_from_state[action] = 'empty'
        return probs['restore_' + next_item] if field_content_equals(copy_from_state, to_state) else 0

    return 0

In [181]:
test_prob = transition_prob(0, ['empty', 'empty', 'empty', 'empty', 'restore', 'red'],
                            ['empty', 'empty', 'empty', 'empty', 'restore', 'red'])

np.testing.assert_equal(test_prob, 1)

In [182]:
def reward(action: int, to_state: List) -> float:
    if to_state[action] == 'empty':
        return rewards_dict[action]

    return 0

In [183]:
def transition_and_reward_matrix():
    transitions = []
    rewards = []

    for action in actions:
        row = []
        col = []
        data = []

        reward_vector = []

        for id_from, from_state in enumerate(states):
            for id_to, to_state in enumerate(states):
                p = transition_prob(action, from_state, to_state)

                if p > 0:
                    row.append(id_from)
                    col.append(id_to)
                    data.append(p)

            reward_vector.append(reward(action, to_state))

        transitions.append(csr_matrix((data, (row, col)), shape=(num_states, num_states)))
        rewards.append(reward_vector)

    return transitions, np.array(rewards).T

In [184]:
P, R = transition_and_reward_matrix()

In [185]:
R

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [186]:
ones = np.zeros((4, num_states))
for i, m in enumerate(P):
    ones[i] = np.sum(np.array(m.toarray()), axis=1)

test = ones[0]
test = np.reshape(test, (32, -1))
np.testing.assert_array_almost_equal(ones, np.ones_like(ones))

In [187]:
np.testing.assert_array_equal(R.shape, (num_states, 4))

In [188]:
algorithms = [mdp.PolicyIteration, mdp.PolicyIterationModified]
discounts = [0.3, 0.5, 0.7, 0.9]

policies = []
for alg in algorithms:
    for d in discounts:
        pass
        # pi = alg(P, R, d)
        # pi.run()

        # policies.append(pi.policy)

In [189]:
pi = mdp.PolicyIteration(P, R, 0.3)
pi.run()

In [190]:
result_policy = pi.policy
print(len(result_policy))

1536


In [191]:
def next_comb_mdp(cur_comb: List, state_index: int, policy: tuple) -> List:
    action = policy[state_index]
    result = cur_comb[:4]
    cur_task = cur_comb[4]
    cur_item = cur_comb[-1]

    if cur_task == 'store':
        result[action] = cur_item
    elif cur_task == 'restore':
        result[action] = 'empty'

    return result

In [192]:
def distance_mdp(warehouse_input: List[tuple], policy: tuple) -> float:
    cur_comb = ['empty', 'empty', 'empty', 'empty']
    total_reward = 0

    for cur_task, cur_item in warehouse_input:
        cur_comb.append(cur_task)
        cur_comb.append(cur_item)

        state_index = states.index(cur_comb)

        cur_comb = next_comb_mdp(cur_comb, state_index, policy)
        total_reward += dist_dict[policy[state_index]]

    return total_reward

In [193]:
def next_comb_greedy(cur_comb: List) -> List:
    result = cur_comb[:4]
    cur_task = cur_comb[4]
    cur_item = cur_comb[-1]

    if cur_task == 'store':
        for action, field in enumerate(result):
            if field == 'empty':
                result[action] = cur_item
                return result, action

    elif cur_task == 'restore':
        for action, field in enumerate(result):
            if field == cur_item:
                result[action] = 'empty'
                return result, action

    return result, 0

In [194]:
def distance_greedy(warehouse_input: List[tuple]) -> float:
    cur_comb = ['empty', 'empty', 'empty', 'empty']
    total_reward = 0

    for cur_task, cur_item in warehouse_input:
        cur_comb.append(cur_task)
        cur_comb.append(cur_item)

        cur_comb, action = next_comb_greedy(cur_comb)
        total_reward += dist_dict[action]

    return total_reward

In [195]:
def data_to_warehouse_input() -> List[tuple]:
    f = open('testingdata.txt', 'r')

    result = []
    line = f.readline()
    while line:
        result.append(tuple(line.split()))
        line = f.readline()

    return result


w_input = data_to_warehouse_input()
print(w_input)

[('store', 'red'), ('store', 'blue'), ('store', 'white'), ('restore', 'blue'), ('restore', 'white'), ('store', 'white'), ('store', 'red'), ('store', 'white'), ('store', 'red'), ('restore', 'red'), ('restore', 'red'), ('store', 'white'), ('store', 'blue'), ('store', 'white'), ('restore', 'red'), ('restore', 'white'), ('restore', 'white'), ('store', 'red'), ('store', 'blue'), ('restore', 'red'), ('store', 'blue'), ('store', 'white'), ('restore', 'white'), ('restore', 'blue'), ('restore', 'white'), ('restore', 'white'), ('restore', 'blue'), ('restore', 'blue'), ('store', 'red'), ('restore', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'white'), ('restore', 'red'), ('restore', 'red'), ('restore', 'red'), ('store', 'red'), ('restore', 'white'), ('restore', 'red'), ('restore', 'white'), ('store', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'white'), ('store', 'blue'), ('restore', 'red'), ('restore', 'red'), ('rest

In [196]:
for policy in policies:
    print(distance_mdp(w_input, policy))

print(distance_mdp(w_input, result_policy))
print(distance_greedy(w_input))

120
192
